In [1]:
!apt-get update && apt-get install -y tesseract-ocr
!pip install pytesseract opencv-python pillow tqdm

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 8.4 MB/s eta 0:00:00m eta 0:00:01m36m0:00:01m


In [2]:
import os
import pandas as pd
import cv2
import pytesseract
from PIL import Image
from tqdm import tqdm

def extract_text_pytesseract(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )
    text = pytesseract.image_to_string(
        gray,
        lang="eng",
        config="--psm 6"
    )
    return text.strip()

# Ventile 1 e 20
df = pd.read_csv("./reddit-memes-virality-prediction/complete_viral_dataset.csv")
df = df[df["Ventile"].isin([1, 20])].reset_index(drop=True)
df["ocr_text"] = ""

# Processando o OCR
root_dir = "reddit-virality-dataset"
for idx, row in tqdm(df.iterrows(), total=len(df)):
    ventile  = row["Ventile"]
    filename = row["Filename"]
    img_path = os.path.join(root_dir, f"Ventile_{ventile}", filename)
    if os.path.isfile(img_path):
        try:
            df.at[idx, "ocr_text"] = extract_text_pytesseract(img_path)
        except Exception:
            df.at[idx, "ocr_text"] = ""
    else:
        df.at[idx, "ocr_text"] = ""
        
df.to_csv("complete_viral_dataset_ventiles1_20_with_ocr.csv", index=False)
print("Novo CSV gerado: complete_viral_dataset_ventiles1_20_with_ocr.csv")

100%|██████████| 1700/1700 [01:40<00:00, 16.89it/s]


Novo CSV gerado: complete_viral_dataset_ventiles1_20_with_ocr.csv
